# 1D J1-J2 model - Inference (with grad clipping) (10k samples)

This notebook is part of the work arXiv:2505.22083 (https://arxiv.org/abs/2505.22083), "Hyperbolic recurrent neural network as the first type of non-Euclidean neural quantum state ansatz". Code written by HLD. 

In this notebook, we load the trained models to check the ground state energy produced by each trained model. The system size is N=50. J1 =1.0, J2 = 0.0, 0.2, 0.5, 0.8. For each (J1,J2) tuple, we train 2 models: EuclGRU and HypGRU and compare their performances. The number of samples used for the estimation is 10000. 

We include results obtained from training with and without gradient clipping (some of the experiments require gradient clipping for stability).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('../../utility')
from j1j2_hyprnn_wf import *
from j1j2_hyprnn_train_loop_gc_gn import *

2025-11-26 16:51:26.447349: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/hldao/opt/anaconda3/lib/python3.9/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.12) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


Training with gradient clipping by global norm of 8.0 (both Euclidean and hyperbolic)
The norm clipping is defined in the Adam optimizer
Num GPUs Available:  0


In [2]:
Ee00 =-21.9721
Ee02 = -20.3150
Ee05 = -18.7500
Ee08 = -20.9842

In [3]:
def define_load_test(wf, numsamples, weight_link, Ee):
    test_samples_before = wf.sample(numsamples,2)
    print(f'The number of samples is {numsamples}')
    test_gs_before = J1J2_local_energies(wf, syssize, J1, J2, Bz, numsamples, test_samples_before, Marshall_sign = True)    
    gs_mean_b = np.round(np.mean(test_gs_before),4)
    gs_var_b = np.round(np.var(test_gs_before),4)
    print(f'Before loading weights, the ground state energy mean and variance are:')
    print(f'Mean E = {gs_mean_b}, var E = {gs_var_b}')
    print('====================================================================')
    wf.model.load_weights(weight_link)
    test_samples_after = wf.sample(numsamples,2)
    test_gs_after = J1J2_local_energies(wf, syssize, J1, J2, Bz, numsamples, test_samples_after, Marshall_sign = True)
    gs_mean_a = np.round(np.mean(test_gs_after),4)
    gs_var_a = np.round(np.var(test_gs_after),4)
    
    #wf.model.summary()
    #print('====================================================================')
    print(f'After loading weights, the ground state energy mean and variance are:')
    print(f'Mean E = {gs_mean_a}, var E = {gs_var_a}')
    print(f'Exact energy is {Ee}')

In [4]:
#COMMON CONFIGS
syssize = 50
nssamples = 10000

J1_ = 1.0
Bz=+0.0*np.ones(syssize)
J1=+J1_*np.ones(syssize) 

# J2=0.0

In [5]:
J2_ = 0.0
J2=+J2_*np.ones(syssize)
fname = '../results_w_grad_clipping/N50_J2=0.0'
print(f'Exact energy for J2=0.0 is E={Ee00}')

Exact energy for J2=0.0 is E=-21.9721


## Inference: Load trained models

In [6]:
print('EUCLIDEAN GRU, 75')
# Clipping by global norm
t1 = time.time()
wf_egru_00 = rnn_eucl_wf(syssize, 'EuclGRU', units=75)
weight_link = f'{fname}/N50_J1=1.0|J2=0.0_EuclGRU_75_ns=50_MsTrue_checkpoint.weights.h5'
define_load_test(wf_egru_00, nssamples, weight_link, Ee00)
t2 = time.time()
print(f'Time taken is t={np.round((t2-t1)/3600,3)} hrs')

EUCLIDEAN GRU, 75
The number of samples is 10000
Before loading weights, the ground state energy mean and variance are:
Mean E = (12.24489974975586-0.004699999932199717j), var E = 0.17820000648498535
After loading weights, the ground state energy mean and variance are:
Mean E = (-21.374500274658203-0.001500000013038516j), var E = 0.3553999960422516
Exact energy is -21.9721
Time taken is t=0.522 hrs


In [7]:
print('HYPERBOLIC GRU, 60')
#Clipping by value
t1 = time.time()
wf_hgru_00_60 = rnn_hyp_wf(syssize, 'HypGRU', 'hyp', 'id', units=60)
weight_link = f'{fname}/N50_J1=1.0|J2=0.0_HypGRU_60_id_hyp_ns=50_MsTrue_gcv_checkpoint.weights.h5'
define_load_test(wf_hgru_00_60, nssamples, weight_link, Ee00)
t2 = time.time()
print(f'Time taken is t={np.round((t2-t1)/3600,3)} hrs')

HYPERBOLIC GRU, 60
The number of samples is 10000
Before loading weights, the ground state energy mean and variance are:
Mean E = (11.657999992370605-0.0142000000923872j), var E = 1.8072999715805054
After loading weights, the ground state energy mean and variance are:
Mean E = (-21.11079978942871-0.005200000014156103j), var E = 1.1464999914169312
Exact energy is -21.9721
Time taken is t=0.899 hrs


# J2 = 0.2

In [8]:
J2_ = 0.2
J2=+J2_*np.ones(syssize)
fname = '../results_w_grad_clipping/N50_J2=0.2'
print(f'Exact energy for J2={J2_} is E={Ee02}')

Exact energy for J2=0.2 is E=-20.315


## Inference: Load trained models

In [9]:
#EUCLGRU_75
print('EUCLIDEAN GRU, 75')
t1 = time.time()
wf_egru_02 = rnn_eucl_wf(syssize, 'EuclGRU', units=75)
weight_link =f'{fname}/N50_J1=1.0|J2=0.2_EuclGRU_75_ns=50_MsTrue_checkpoint.weights.h5'
define_load_test(wf_egru_02, nssamples, weight_link, Ee02)
t2 = time.time()
print(f'Time taken is t={np.round((t2-t1)/3600,3)} hrs')

EUCLIDEAN GRU, 75
The number of samples is 10000
Before loading weights, the ground state energy mean and variance are:
Mean E = (14.638999938964844-0.00570000009611249j), var E = 0.27559998631477356
After loading weights, the ground state energy mean and variance are:
Mean E = (-19.42289924621582-0.0003000000142492354j), var E = 0.14839999377727509
Exact energy is -20.315
Time taken is t=0.984 hrs


In [10]:
#HYPGRU_75
print('HYPERBOLIC GRU, 75')
t1 = time.time()
wf_hgru_02_75 = rnn_hyp_wf(syssize, 'HypGRU', 'hyp', 'id', units=75)
weight_link = f'{fname}/N50_J1=1.0|J2=0.2_HypGRU_75_id_hyp_ns=50_MsTrue_checkpoint.weights.h5'
define_load_test(wf_hgru_02_75, nssamples, weight_link, Ee02)
t2 = time.time()
print(f'Time taken is t={np.round((t2-t1)/3600,3)} hrs')

HYPERBOLIC GRU, 75
The number of samples is 10000
Before loading weights, the ground state energy mean and variance are:
Mean E = (12.546600341796875+0.02410000003874302j), var E = 6.05649995803833
After loading weights, the ground state energy mean and variance are:
Mean E = (-19.913299560546875-0.00139999995008111j), var E = 0.4180000126361847
Exact energy is -20.315
Time taken is t=1.648 hrs


# J2 = 0.5

In [5]:
J2_ = 0.5
J2=+J2_*np.ones(syssize)
fname = '../results_w_grad_clipping/N50_J2=0.5'
print(f'Exact energy for J2={J2_} is E={Ee05}')

Exact energy for J2=0.5 is E=-18.75


## Inference: Load trained models

### EuclGRU 75

In [11]:
print('EUCLIDEAN GRU, 75')
t1 = time.time()
wf_egru_05 = rnn_eucl_wf(syssize, 'EuclGRU', units=75, seed = 121)
weight_link = f'{fname}/N50_J1=1.0|J2=0.5_EuclGRU_75_ns=50_MsTrue_checkpoint.weights.h5'
define_load_test(wf_egru_05, nssamples, weight_link, Ee05)
t2 = time.time()
print(f'Time taken is t={np.round((t2-t1)/3600,3)} hrs')

EUCLIDEAN GRU, 75
The number of samples is 10000
Before loading weights, the ground state energy mean and variance are:
Mean E = (18.045799255371094+0.0071000000461936j), var E = 0.8644000291824341
After loading weights, the ground state energy mean and variance are:
Mean E = (-18.662799835205078+0.0027000000700354576j), var E = 0.2773999869823456
Exact energy is -18.75
Time taken is t=0.85 hrs


### HypGRU 70

In [7]:
print('HYPERBOLIC GRU, 70')
t1 = time.time()
wf_hgru_05_70 = rnn_hyp_wf(syssize, 'HypGRU', 'hyp', 'id', units=70, seed = 121)
weight_link = f'{fname}/N50_J1=1.0|J2=0.5_HypGRU_70_id_hyp_ns=50_ngc_MsTrue_checkpoint.weights.h5'
define_load_test(wf_hgru_05_70, nssamples, weight_link, Ee05)
t2 = time.time()
print(f'Time taken is t={np.round((t2-t1)/3600,3)} hrs')

HYPERBOLIC GRU, 70
The number of samples is 10000
Before loading weights, the ground state energy mean and variance are:
Mean E = (16.16309928894043+0.026799999177455902j), var E = 3.3636999130249023
After loading weights, the ground state energy mean and variance are:
Mean E = (-18.667999267578125+0.000699999975040555j), var E = 0.22280000150203705
Exact energy is -18.75
Time taken is t=1.036 hrs


# J2 =0.8

In [5]:
J2_ = 0.8
J2=+J2_*np.ones(syssize)
fname = '../results_w_grad_clipping/N50_J2=0.8'
print(f'Exact energy for J2={J2_} is E={Ee08}')

Exact energy for J2=0.8 is E=-20.9842


## Inference: Load trained models

In [16]:
print('EUCLIDEAN GRU, 75')
t1 = time.time()
wf_egru_08 = rnn_eucl_wf(syssize, 'EuclGRU', units=75)
weight_link = f'{fname}/N50_J1=1.0|J2=0.8_EuclGRU_75_ns=50_MsTrue_checkpoint.weights.h5'
define_load_test(wf_egru_08, nssamples, weight_link, Ee08)
t2 = time.time()
print(f'Time taken is t={np.round((t2-t1)/3600,3)} hrs')

EUCLIDEAN GRU, 75
The number of samples is 10000
Before loading weights, the ground state energy mean and variance are:
Mean E = (21.82110023498535-0.008799999952316284j), var E = 0.8420000076293945
After loading weights, the ground state energy mean and variance are:
Mean E = (-19.209199905395508+0.008799999952316284j), var E = 1.75
Exact energy is -20.9842
Time taken is t=1.314 hrs


In [17]:
print('HYPERBOLIC GRU, 75')
t1 = time.time()
wf_hgru_08_75 = rnn_hyp_wf(syssize, 'HypGRU', 'hyp', 'id', units=75)
weight_link = f'{fname}/N50_J1=1.0|J2=0.8_HypGRU_75_id_hyp_ns=50_gn=8_MsTrue_checkpoint.weights.h5'
define_load_test(wf_hgru_08_75, nssamples, weight_link, Ee08)
t2 = time.time()
print(f'Time taken is t={np.round((t2-t1)/3600,3)} hrs')

HYPERBOLIC GRU, 75
The number of samples is 10000
Before loading weights, the ground state energy mean and variance are:
Mean E = (18.764999389648438+0.03550000116229057j), var E = 10.951299667358398
After loading weights, the ground state energy mean and variance are:
Mean E = (-19.61910057067871+0.010999999940395355j), var E = 2.072499990463257
Exact energy is -20.9842
Time taken is t=1.86 hrs
